### Mantle rheology

The goal of this notebook is to walk through mantle rheology if the form of dislocation and diffusion creep.

These contents are included(todo):

* Introduce flow law with the parameters for olivine defined in Hirth & Kohlstedt 2003
* An update on the parameters with new results (reference: explain_update_modHK03_rheology.pdf)

In [ ]:
import numpy as np
from scipy.special import erf
from matplotlib import pyplot as plt
from matplotlib import gridspec

R = 8.314
year = 365.0 * 24.0 * 3600.0
eta_min = 1e18
eta_max = 1e25

#### Step 1: Define the parameters in flow law and view and example

##### 1.1 Form of the flow law

$\eta_{df, ds} = F \left(\frac{d^p}{A C^r_{OH}} \right)\dot{\epsilon_E}^{\frac{1-n}{n}}exp\left(\frac{E+P_{lc}V}{nRT}\right)$

Here F is a effective coefficient to account for the differences in the conditions in experiments.

In [ ]:
def CreepRheology(creep_type, strain_rate, P, T, d=None, Coh=None, **kwargs):
    """
    def CreepRheology(creep_type, strain_rate, P, T, d, Coh):

    Calculate viscosity by flow law in form of (strain_rate)**(1.0 / n - 1) * (B)**(-1.0 / n) * np.exp((E + P * V) / (n * R * T))
    Units:
     - P: Pa
     - T: K
     - d: mu m
     - Coh: H / 10^6 Si
     - Return value: Pa*s
    Pay attention to pass in the right value, this custom is inherited
    """
    A = creep_type['A']
    p = creep_type['p']
    r = creep_type['r']
    n = creep_type['n']
    E = creep_type['E']
    V = creep_type['V']
    # compute value of F(pre factor)
    use_effective_strain_rate = kwargs.get('use_effective_strain_rate', False)
    if use_effective_strain_rate:
        F = 1 / (2**((n-1)/n)*3**((n+2)/2/n))
    else:
        F = 1.0

    if d is None:
        d = creep_type['d']
    if Coh is None:
        Coh = creep_type['Coh']
    # calculate B
    B = A * d**(-p) * Coh**r
    eta = 1/2.0 * F * (strain_rate)**(1.0 / n - 1) * (B)**(-1.0 / n) * np.exp((E + P * V) / (n * R * T)) * 1e6

    return eta






#### 1.2 The example from Hirth and Kohlstedt 2003, table 1

![](./HK03_table1.png)

In [ ]:
# dislocation creep in Hirth & Kohlstedt 2003
HK03_disl = \
    {
        "A": 90,
        "p": 0.0,
        "r": 1.2,
        "n": 3.5,
        "E": 480e3,
        "V": 11e-6,
        "d" : 1e4,
        "Coh" : 1000.0
    }

# diffusion creep in Hirth & Kohlstedt 2003
HK03_diff = \
    {
        "A" : 1.0e6,
        "p" : 3.0,
        "r" : 1.0,
        "n" : 1.0,
        "E" : 335e3,
        "V" : 4e-6,
        "d" : 1e4,
        "Coh" : 1000.0
    }

# modified creep laws from Hirth & Kohlstedt 2003
# for detail, refer to magali's explain_update_modHK03_rheology.pdf file
HK03_wet_mod_diff = \
    {
        # "A" : 10**6.9,  # MPa^(-n-r)*um**p/s
        "A" : 7.1768e6,  # MPa^(-n-r)*um**p/s
        "p" : 3.0,
        "r" : 1.0,
        "n" : 1.0,
        "E" : 375e3,
        "V" : 23e-6,
        "d" : 1e4,
        "Coh" : 1000.0,
        "wet": 1.0  # I use this to mark this is a wet rheology, so I need to account for V and E for water later.
    }

HK03_wet_mod_disl = \
    {
        "A" : 10**2.65,
        "p" : 0.0,
        "r" : 1.0,
        "n" : 3.5,
        "E" : 520e3,
        "V" : 24e-6,
        "d" : 1e4,
        "Coh" : 1000.0,
        "wet" : 1.0
    }


To make sure our computation is valid, we can calculate the two examples given under table 1 directly.

The only difference is we are computing the viscosity, while the examples compute stress. In this way, we will mutiply our viscosity with 2 times the strain rate to get the stress to compare.

Note that this case shows the viscosity mantle is "highly deformed" even under 0.3 Mpa, as the strain rate from the dislocation creep already reaches $1 \times 10^{-12} s-1$

In [ ]:
pressure = 1e9
temperature = 1673.0 # K
# diffusion creep
strain_rate = 7.8e-15
eta_diff = CreepRheology(HK03_diff, strain_rate, pressure, temperature)
stress = 2.0 * strain_rate * eta_diff
print("At strain rate = %.4e s-1, diffusion creep (COH = 1000) from Hirth & Kohlstedt 2003 gives viscosity = %.4e Pa*s, stress = %.4e Mpa"\
     % (strain_rate, eta_diff, stress/1e6))
# dislocation creep
strain_rate = 2.5e-12
eta_disl = CreepRheology(HK03_disl, strain_rate, pressure, temperature)
stress = 2.0 * strain_rate * eta_disl
print("At strain rate = %.4e s-1, dislocation creep (COH = 1000) from Hirth & Kohlstedt 2003 gives viscosity = %.4e Pa*s, stress = %.4e Mpa"\
     % (strain_rate, eta_disl, stress/1e6))

##### 1.3 Plot the  mantle rheologic profile with a temperature & pressure profile

Here, we first generate a temperature profile and a pressure profile.

The temperature profile is generated from the Half-space cooling model and a mantle adiabat.

The pressure profile, on the other hand, takes care of the density change due to thermal expansivity on the temperature profile.
The viscosity profile, is then computed on both T and P assuming some grain size.

##### Note on the Hirth & Kolstedt 2003 rheology

At this step, we apply the F factor for the rheology, as the experiments were performed with a simple-shear sense.

We can observe that their choice of wet rheology of constant $C_{OH}$ would produce comparable values between the diffusion and the dislocation creep.
Moreover, the order of magnitude is aounnd 1e19 - 1e20 for the upper mantle average ($ \dot{\epsilon} = 1 \times 10^{-15}$).
(future: discuss with Max and Magali)

In [ ]:

def pressure_from_lithostatic(z,Tad):
    '''
    A lithostatic pressure profile
    Inputs:
	z (float) - depth in m
	Tad (float) - adiabatic temperature
    '''
    # Density Profile
    refrho = 3300  # kg/m^3
    refT = 1673        # K
    alpha = 3.1e-5  # 1/K
    g = 9.81 # m/s^2
    density = refrho*(1-alpha*(Tad-refT))

    # start loop at 1 because P[0] = 0
    dz = z[1]-z[0]
    P = np.zeros(np.size(z))
    for i in range(1, np.size(z)):
        P[i] = P[i-1] + 0.5*(density[i]+density[i-1])*g*dz
    return P


def temperature_halfspace(z, t, **kwargs):
    '''
    temperature from a half-space cooling model
    Inputs:
	z (float) - depth (m)
	t - age (s)
    kwargs (dict):
        Tm (float) - mantle temperature
    '''
    # Physical constants
    kappa = 1e-6  # thermal diffusivity (m^2/s)
    T_s = 273  # surface temperature (K)
    T_m = kwargs.get("Tm", 1673) # mantle temperature (K)
    T = T_s + (T_m - T_s)*erf(z/(2*np.sqrt(kappa*t)))
    return T

strain_rate = 1e-15
zs = np.arange(0.0, 660.0, 1.0) * 1e3
age = 80e6 * year
Tad_surface = 1673.0 
mad = 0.5715  # deg/km, approximating adiabat to match T(CMB) in Aspect
Tads = Tad_surface + mad*(zs/1e3)
Ts = np.zeros(zs.shape)  # temperature, combine HS model with adiabat
for i in range(zs.size):
    z = zs[i]
    Tad = Tads[i]
    Ts[i] = temperature_halfspace(z, age, Tm = Tad)
Ps = pressure_from_lithostatic(zs, Ts)  # pressure, using the computed temperature
# compute viscosities
etas_disl = CreepRheology(HK03_disl, strain_rate, Ps, Ts, use_effective_strain_rate=True)
etas_diff = CreepRheology(HK03_diff, strain_rate, Ps, Ts, use_effective_strain_rate=True)
# plot figures
fig = plt.figure(figsize=(5, 15))
ax = fig.add_subplot(3, 1, 1)  # plot pressure
ax.plot(Ps/1e9, zs/1e3, 'b')
ax.set_xlabel("Pressure (GPa)")
ax.set_ylabel("Depth (km)")
ax.invert_yaxis()
ax = fig.add_subplot(3, 1, 2)  # plot temperature
ax.plot(Ts, zs/1e3, 'r')
ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Depth (km)")
ax.invert_yaxis()
ax = fig.add_subplot(3, 1, 3)  # plot viscosity
ax.semilogx(etas_disl, zs/1e3, 'g', label="dislocation creep (HK03)")
ax.semilogx(etas_diff, zs/1e3, 'k', label="diffusion creep (HK03)")
ax.set_xlim([eta_min, eta_max])  # set x limit
ax.set_xlabel("viscosity (Pa*s))")
ax.set_ylabel("Depth (km)")
ax.set_title("Viscosity with limits [%.1e, %.1e]" % (eta_min, eta_max))
ax.legend()
ax.invert_yaxis()
fig.tight_layout()
# separately plot viscosity without limits
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.semilogx(etas_disl, zs/1e3, 'g', label="dislocation creep (HK03)")
ax.semilogx(etas_diff, zs/1e3, 'k', label="diffusion creep (HK03)")
ax.set_xlabel("viscosity (Pa*s))")
ax.set_ylabel("Depth (km)")
ax.legend()
ax.invert_yaxis()
ax.set_title("Viscosity without limits")
fig.tight_layout()

#### Step 2: derive a upper mantle viscosity profile for geodynamics

This are 2 things need to be done after Step 1 to reach a profile:
* 1. Apply yielding in the shallower parts.
* 2. At each depth, combine the computation of different flow laws.

There are both physical reason and numerical reason to have a yielding scheme at shallower depth.
Physically, the shallower part has much lower temperature, thus subjected to brittle deformation.
Numerically, the variation in viscosity a numerical code could handle is always limited (at best 7-8 orders).
This point is showcased in the previous step with the plot of the raw values of viscosities with no limits, where the variation could be 10s of orders.

##### 2.1 Define brittle failure criterion

The failure of the brittle crust and lithosphere is controlled by frictional and/or fracture strength. For strength-depth profiles, the frictional strength is typically used with the assumption that pre-existing faults limit the effective brittle strength. Therefore, we can use the empirically-defined Byerlee's law for fricational sliding to estimate the brittle strength of the crust and lithosphere:

$\tau = 0.85\sigma_n$, for $\sigma_n < 200$ MPa

$\tau = 60 + 0.6\sigma_n$, for $200 < \sigma_n < 1700$ MPa

where $\tau$ is the shear strength, $\sigma_n$ is the normal stress on a fault (both in MPa). In using this to estimate plate strength, I will assume that $\sigma_n$ is given by the lithostatic pressure.

Note that this is the starting point for determining brittle strength, but this ignores the effect of pore fluid pressure on strength and does not account for the different strength of the crust in extension, compression or strike-slip environments ($\sigma_n$ depends on type of deformation). For more on these effects, see *Implications of hydrostatic pore pressures and high crustal strength for the deformation of intraplate lithosphere*, Mark D. Zoback and John Townend, Tectonophysics, **336**, 19-30, (2001). 

In [ ]:
def Byerlee(P):
    '''
    byerlee's law for yielding
    Inputs:
        P (pressure, Pa) - lithostatic pressure
    '''
    if type(P) == float:
        sigma_n = P  # future: pore pressure
        if P < 200e6:
            tau = 0.85 * P
        else:
            tau = 0.6 * P + 60e6
    elif type(P) == np.ndarray:
        tau = np.zeros(P.shape)
        mask = (P < 200e6)
        tau[mask] = 0.85 * P[mask]
        tau[~mask] = 0.6 * P[~mask] + 60e6
    else:
        raise TypeError("Wrong type of entry")
    return tau
tauBs = Byerlee(Ps)  # compute the Byerlee stresses
etas_B = tauBs / 2 / strain_rate  # equivalent viscosity from the Byerlee's law

# plot the Byerlee yielding cryteria on top of the profile from last step.
fig = plt.figure(figsize=(5, 10))
gs = gridspec.GridSpec(2, 1)
ax = fig.add_subplot(gs[0, :])  # plot viscosity
ax.semilogx(etas_disl, zs/1e3, 'g', label="dislocation creep (HK03)")
ax.semilogx(etas_diff, zs/1e3, 'k', label="diffusion creep (HK03)")
ax.semilogx(etas_B, zs/1e3, 'r', label="Byerlee's law")
ax.set_xlim([eta_min, eta_max])  # set x limit
ax.set_xlabel("viscosity (Pa*s))")
ax.set_ylabel("Depth (km)")
ax.set_title("Viscosity with limits [%.1e, %.1e]" % (eta_min, eta_max))
ax.legend()
ax.invert_yaxis()

##### Step 2.2 Use a composite scheme to get the profile

One way is to use a minimum scheme:
$$\eta_{comp} = min\left(\eta_{diffusion}, \eta_{dislocation}, \eta_{Byerlee}\right)$$

Another way around is to apply harmonic averaging (aka geometric averaging)
$$\frac{1}{\eta_{comp}} = (\frac{1}{\eta_{diffusion}} + \frac{1}{\eta_{dislocation}} + \frac{1}{\eta_{Byerlee}}) / n$$

These two schemes agrees with each other with a factor of 1.5.

In [ ]:
def harmonic_average(*args):
    '''
    Harmonic averaging of entries
    Inputs:
        args (list): the averaging will be performed on
                    args[0], args[1], args[2] ...
    '''
    if type(args[0]) == float:
        aver = 0.0
        for arg in args:
            assert(abs(arg) > 1e-15)
            aver += 1.0 / arg
            aver = len(args) / aver
    elif type(args[0]) == np.ndarray:
        length = len(args)
        _size = args[0].size
        aver = np.zeros(_size)
        for arg in args:
            assert(arg.size == _size)
            assert(min(abs(arg)) > 1e-15)
            aver += 1.0 / arg
        aver = length / aver
    else:
        raise TypeError("harmonic_average: inputs must by float or ndarray")
    return aver


etas_comp_min = np.minimum(etas_diff, etas_disl, etas_B)
etas_comp_harmonic = harmonic_average(etas_diff, etas_disl, etas_B)
etas_comp_diff = etas_comp_harmonic / etas_comp_min

fig = plt.figure(figsize=(10, 5))
gs = gridspec.GridSpec(1, 2)
ax = fig.add_subplot(gs[0, 0])  # plot viscosity
ax.semilogx(etas_comp_min, zs/1e3, 'b', label="Composite (min)")
ax.semilogx(etas_comp_harmonic, zs/1e3, 'r', label="Composite (harmonic)")
ax.set_xlim([eta_min, eta_max])  # set x limit
ax.set_xlabel("viscosity (Pa*s))")
ax.set_ylabel("Depth (km)")
ax.legend()
ax.invert_yaxis()
ax = fig.add_subplot(gs[0, 1])  # plot difference
ax.plot(etas_comp_diff, zs/1e3, 'b', label="Difference between min and harmonic")
ax.invert_yaxis()
fig.tight_layout()

##### plot the deformation mechanism

Here I ploted deformation mechanism at 800, 1300 C, respectively.
Note the colorbar in these 2 plots are different.

At lower temperature, the deformation is determined by diffusion, dislocation and Byerlee's law, together.
While at higher temperature, the Byerlee's yielding criteria is not triggered.

Compared to the diffusion creep, the dislocation creep tends to occupy the region with larger grain size and larger strain rate.

In [ ]:
# todo
ds = 10 ** np.linspace(0, 6, 200)
strain_rates = 10 ** np.linspace(-20, 0, 100)
Ds, StrainRates = np.meshgrid(ds, strain_rates)
# T = 800 C, 200km
T0 = 800.0 + 273.0
P0 = 6.6e9
EtaDiffs =  CreepRheology(HK03_diff, StrainRates, P0, T0, Ds, use_effective_strain_rate=True)
SigmaDiffs = 2 * EtaDiffs * StrainRates
EtaDisls =  CreepRheology(HK03_disl, StrainRates, P0, T0, Ds, use_effective_strain_rate=True)
SigmaDisls = 2 * EtaDisls * StrainRates
EtaBs = np.ones(Ds.shape) * Byerlee(P0) / 2 / StrainRates
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 3)
ax = fig.add_subplot(gs[0, 0])  # diffusion creep
h = ax.pcolormesh(np.log10(Ds), np.log10(StrainRates), np.log10(EtaDiffs))
ax.set_xlabel("log(grain size (mu m))")
ax.set_ylabel("log(strain rate (s^-1))")
fig.colorbar(h, ax=ax, label='Eta_diff (Pa*s)')
ax.set_title("Diffusion creep")
ax = fig.add_subplot(gs[0, 1])  # dislocation creep
h = ax.pcolormesh(np.log10(Ds), np.log10(StrainRates), np.log10(EtaDisls))
ax.set_xlabel("log(grain size (mu m))")
ax.set_ylabel("log(strain rate (s^-1))")
fig.colorbar(h, ax=ax, label='Eta_diff (Pa*s)')
ax.set_title("Dislocaion creep")
DFMechanisms = np.ones(Ds.shape)  # deformation mechanism
mask2 = (EtaDisls < EtaDiffs) & (EtaDisls < EtaBs)
mask3= (EtaBs < EtaDiffs) & (EtaBs < EtaDisls)
DFMechanisms[mask2] = 2.0
DFMechanisms[mask3] = 3.0
ax = fig.add_subplot(gs[0, 2])
h = ax.pcolormesh(np.log10(Ds), np.log10(StrainRates), DFMechanisms)
ax.set_xlabel("log(grain size (mu m))")
ax.set_ylabel("log(strain rate (s^-1))")
fig.colorbar(h, ax=ax, label='Deformation Mechanism')
ax.set_title("Deformation mechanism (1-diff, 2-disl, 3-Byerlee), T = %.1f K, P = %.1f GPa" % (T0, P0/1e9))
# T = 1300 C, 200 km
T1 = 1300.0 + 273.0
P1 = 6.6e9
EtaDiffs =  CreepRheology(HK03_diff, StrainRates, P1, T1, Ds, use_effective_strain_rate=True)
SigmaDiffs = 2 * EtaDiffs * StrainRates
EtaDisls =  CreepRheology(HK03_disl, StrainRates, P1, T1, Ds, use_effective_strain_rate=True)
SigmaDisls = 2 * EtaDisls * StrainRates
EtaBs = np.ones(Ds.shape) * Byerlee(P1) / 2 / StrainRates
ax = fig.add_subplot(gs[1, 0])  # diffusion creep
h = ax.pcolormesh(np.log10(Ds), np.log10(StrainRates), np.log10(EtaDiffs))
ax.set_xlabel("log(grain size (mu m))")
ax.set_ylabel("log(strain rate (s^-1))")
fig.colorbar(h, ax=ax, label='Eta_diff (Pa*s)')
ax.set_title("Diffusion creep")
ax = fig.add_subplot(gs[1, 1])  # dislocation creep
h = ax.pcolormesh(np.log10(Ds), np.log10(StrainRates), np.log10(EtaDisls))
ax.set_xlabel("log(grain size (mu m))")
ax.set_ylabel("log(strain rate (s^-1))")
fig.colorbar(h, ax=ax, label='Eta_diff (Pa*s)')
ax.set_title("Dislocaion creep")
DFMechanisms = np.ones(Ds.shape)  # deformation mechanism
mask2 = (EtaDisls < EtaDiffs) & (EtaDisls < EtaBs)
mask3= (EtaBs < EtaDiffs) & (EtaBs < EtaDisls)
DFMechanisms[mask2] = 2.0
DFMechanisms[mask3] = 3.0
ax = fig.add_subplot(gs[1, 2])
h = ax.pcolormesh(np.log10(Ds), np.log10(StrainRates), DFMechanisms)
ax.set_xlabel("log(grain size (mu m))")
ax.set_ylabel("log(strain rate (s^-1))")
fig.colorbar(h, ax=ax, label='Deformation Mechanism')
ax.set_title("Deformation mechanism (1-diff, 2-disl, 3-Byerlee), T = %.1f K, P = %.1f GPa" % (T1, P1/1e9))
fig.tight_layout()

### Read aspect depth_average file

I would be called later with the class function, so the class could know about the temperature and pressure profile

#### compute a lower mantle rheology for aspect